## Symengineの検証

In [1]:
import symengine as sm
import sympy as smp
import numpy as np
import itertools
import math

In [2]:
sm.init_printing()

In [3]:
def g(x):
    return sm.tanh(x)

def dg(x):
    return 1 - sm.tanh(x) * sm.tanh(x)

def dg_np(x):
    return 1 - np.tanh(x) * np.tanh(x)

def ddg(x):
    return 2 * (sm.tanh(x) * sm.tanh(x) - 1) * sm.tanh(x)

In [4]:
# https://stackoverflow.com/questions/22857162/multivariate-taylor-approximation-in-sympy

#import scipy.special

def mv_series(function_expression, 
              variable_list, 
              evaluation_point, 
              degree):
    
    n_var = len(variable_list)
    
    # list of tuples with variables and their evaluation_point coordinates, ready for the subs() method
    #point_coordinates = [(i, j) for i, j in (zip(variable_list, evaluation_point))]
    point_coordinates = {i : j for i, j in zip(variable_list, evaluation_point) }
    
    # list with exponents of the partial derivatives
    deriv_orders = list(itertools.product(range(degree + 1), repeat=n_var))
    
    # Discarding some higher-order terms
    deriv_orders = [deriv_orders[i] for i in range(len(deriv_orders)) if sum(deriv_orders[i]) <= degree]
    n_terms = len(deriv_orders)
    
    # same as before, but now ready for the diff() method
    deriv_orders_as_input = [list(sum(list(zip(variable_list, deriv_orders[i])), ())) for i in range(n_terms)]
    
    polynomial = 0
    for i in range(n_terms):
        partial_derivatives_at_point = function_expression.diff(*deriv_orders_as_input[i]).subs(point_coordinates)
        # e.g. df/(dx*dy**2) evaluated at (x0,y0)
        
        #denominator = sm.prod([sm.factorial(j) for j in deriv_orders[i]])    
        #denominator = np.prod([scipy.special.factorial(j) for j in deriv_orders[i]]) 
        denominator = np.prod([math.factorial(j) for j in deriv_orders[i]])
        # e.g. (1! * 2!)
        
        distances_powered = np.prod([(sm.Matrix(variable_list) - sm.Matrix(evaluation_point))[j] ** deriv_orders[i][j] for j in range(n_var)])  
        # e.g. (x-x0)*(y-y0)**2
        
        polynomial += partial_derivatives_at_point / denominator * distances_powered
    return polynomial


In [5]:
x_dim = 3
r_dim = 4
#r_dim = 100

In [6]:
# Symbols

xs = [sm.Symbol(f'x_{i}') for i in range(x_dim)]
x_dots = [sm.Symbol('\dot{x}_' + f'{i}') for i in range(x_dim)]

x = sm.Matrix([[x_i] for x_i in xs])
x_dot = sm.Matrix([[x_dot_i] for x_dot_i in x_dots])

In [7]:
# Concrete values
def calc_scale_with_spectral_radius(mat, rho):
    cur_spectral_radius = max(abs(np.linalg.eigvals(mat)))
    return rho / cur_spectral_radius

rng = np.random.RandomState(0)

r_s_np = rng.uniform(low=-0.5, high=0.5, size=(r_dim, 1))
r_s_c = sm.Matrix(r_s_np.tolist())

x_s_np = np.zeros((x_dim, 1))
x_s_c = sm.Matrix(x_s_np.tolist())

B_scale = 0.1
B_np = (rng.uniform(low=-1.0, high=1.0, size=(r_dim, x_dim)) * B_scale)
B_c = sm.Matrix(B_np.tolist())

A_np = rng.uniform(low=-1.0, high=1.0, size=(r_dim, r_dim))
sparsify = np.vectorize(lambda v: v if rng.random() < 0.05 else 0.0)
A_np = sparsify(A_np)

rho = 0.01
A_scale = calc_scale_with_spectral_radius(A_np, rho)
print(f'A_scale={A_scale}')
#A_np = A_np * A_scale

A_c = sm.Matrix(A_np.tolist())

d_np = np.zeros((r_dim, 1))

#d_c = sm.Matrix().tolist())                
#d_s_c = A_c * r_s_c + d_c

d_s_np = A_np @ r_s_np + d_np

d_s_c = sm.Matrix(d_s_np.tolist())

ArBxd_np = A_np @ r_s_np + B_np @ x_s_np + d_s_np

#ArBxd_c = A_c * r_s_c + B_c * x_s_c + d_s_c
# 要素積のためにshapeを合わせておく
#ArBxd_c_rep = sm.Matrix.hstack(*[ArBxd_c for _ in range(r_dim)])

#A_s_c = sm.hadamard_product(ArBxd_c_rep.applyfunc(dg), A_c) - sm.eye(r_dim)

# numpyはshapeがあっていなくてもアダマール積が可能なので利用
A_s_np = dg_np(ArBxd_np) * A_np - np.eye(r_dim)

A_s_inv_np = np.linalg.inv(A_s_np)

A_s_inv_c = sm.Matrix(A_s_inv_np.tolist())

gamma_c = 100

A_scale=inf


/var/folders/g8/lw05g79s3yv331w4vf305fkh0000gn/T/ipykernel_52048/3414892641.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  return rho / cur_spectral_radius


In [8]:
B       = B_c
d_s     = d_s_c
r_s     = r_s_c
A_s_inv = A_s_inv_c
A       = A_c
gamma   = gamma_c

In [9]:
expr0 = (B * x + d_s).applyfunc(dg).multiply_elementwise(A * r_s)
expr1 = (B * x + d_s).applyfunc(g)
expr2 = (B * x + d_s).applyfunc(ddg).multiply_elementwise(A * r_s)
expr3 = (B * x + d_s).applyfunc(dg)

expr2_3 = (expr2 - expr3).multiply_elementwise(B * x_dot)

expr = A_s_inv * (expr0 - expr1) + (1 / gamma) * (A_s_inv * A_s_inv) * expr2_3

x_zero = [0] * x_dim
x_dot_zero = [0] * x_dim

degree = 2

In [10]:
%%time

coeff_dicts = []

for r_index in range(r_dim):
    r_ex = mv_series(expr[r_index,0], 
                     xs + x_dots, 
                     x_zero + x_dot_zero, degree)
    
    #print(r_ex)
    #print(r_ex.as_coefficients_dict())
             
    # ここはsympyを使う
    #p_c = smp.poly(r_ex, xs + x_dots)
    p_c = smp.poly(r_ex)
    print(p_c.args[1:])
    # TODO: coeffの順番入れ替え
    coeff_dicts.append(p_c.as_dict())

(\dot{x}_0, \dot{x}_1, \dot{x}_2, x_0, x_1, x_2)
(\dot{x}_0, \dot{x}_1, \dot{x}_2, x_0, x_1, x_2)
(\dot{x}_0, \dot{x}_1, \dot{x}_2, x_0, x_1, x_2)
(\dot{x}_0, \dot{x}_1, \dot{x}_2, x_0, x_1, x_2)
CPU times: user 53.5 ms, sys: 4.43 ms, total: 57.9 ms
Wall time: 59.3 ms


In [11]:
coeff_dicts

[{(0, 0, 0, 0, 0, 0): 0.00192404757385711,
  (0, 0, 0, 0, 0, 1): -0.0270311237029534,
  (0, 0, 0, 0, 0, 2): -1.17734096674772e-6,
  (0, 0, 0, 0, 1, 0): 0.0866494209249828,
  (0, 0, 0, 0, 1, 1): 5.50422820693555e-6,
  (0, 0, 0, 0, 2, 0): -6.43325277249881e-6,
  (0, 0, 0, 1, 0, 0): 0.0325056338058550,
  (0, 0, 0, 1, 0, 1): -2.88031777369224e-6,
  (0, 0, 0, 1, 1, 0): 6.73293751880959e-6,
  (0, 0, 0, 2, 0, 0): -1.76164567269855e-6,
  (0, 0, 1, 0, 0, 0): 0.000413341732326764,
  (0, 0, 1, 0, 0, 1): 2.35468193349544e-8,
  (0, 0, 1, 0, 1, 0): -5.50422820693554e-8,
  (0, 0, 1, 1, 0, 0): 2.88031777369224e-8,
  (0, 1, 0, 0, 0, 0): -0.00143546107993353,
  (0, 1, 0, 0, 0, 1): -5.50422820693554e-8,
  (0, 1, 0, 0, 1, 0): 1.28665055449976e-7,
  (0, 1, 0, 1, 0, 0): -6.73293751880961e-8,
  (1, 0, 0, 0, 0, 0): -0.000805806308214608,
  (1, 0, 0, 0, 0, 1): 2.88031777369224e-8,
  (1, 0, 0, 0, 1, 0): -6.73293751880959e-8,
  (1, 0, 0, 1, 0, 0): 3.52329134539711e-8},
 {(0, 0, 0, 0, 0, 1): -0.0233116962348445,
